In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [4]:
%cd "/content/drive/MyDrive/chest-xray-pneumonia"

/content/drive/MyDrive/chest-xray-pneumonia


In [1]:
# Instalar y tener acceso a kaggle mediante
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
# Descarga del dataset Chest X-Ray Images (Pneumonia)
# link: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
!kaggle datasets download paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip

100% 2.29G/2.29G [00:22<00:00, 188MB/s]
100% 2.29G/2.29G [00:22<00:00, 110MB/s]


In [3]:
# Eliminado de ficheros y carpetas innecesarias
!rm chest-xray-pneumonia.zip
!rm -rf chest_xray/__MACOSX/
!rm -rf chest_xray/chest_xray/